In [9]:
import pandas as pd

from opencage.geocoder import OpenCageGeocode
from geopy.geocoders import Nominatim

import reverse_geocoder as rg
import pprint

geolocator = Nominatim(user_agent="HUR")


In [10]:
#test the geolocator works
location = geolocator.reverse('25.966660,-80.246790')
print(location.address)

20956, Northwest 27th Avenue, Lake Lucerne, Miami Gardens, Miami-Dade County, Florida, 33056, United States


In [11]:
# read-in csv file Atlantic hurricane data
HUR_data=pd.read_csv('Atlantic_HURDAT.csv', sep=',')
HUR_data.head(3)

HUR_df = pd.DataFrame(HUR_data)


In [12]:
#print the data
HUR_df.head(2)

,index,id,name,date,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,...,34_kt_sw,34_kt_nw,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw
0,0,AL011851,UNNAMED,1851-06-25 00:00:00,NaN,HU,28.0,-94.8,80,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AL011851,UNNAMED,1851-06-25 06:00:00,NaN,HU,28.0,-95.4,80,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# convert just columns "a" and "b"
HUR_df[['latitude', 'longitude']] = HUR_df[['latitude', 'longitude']].apply(pd.to_numeric)

HUR_df = HUR_df.infer_objects()

HUR_df.dtypes

HUR_df['lat/long'] = HUR_df['latitude'].astype(str) + "," + HUR_df['longitude'].astype(str) 

In [14]:
HUR_df['lat/long'].head(3)

0    28.0,-94.8
1    28.0,-95.4
2    28.0,-96.0
Name: lat/long, dtype: object

In [16]:
#drop any nulls
Subset_df = HUR_df.dropna(subset=['record_identifier'])
Subset_df.tail()

,index,id,name,date,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,...,34_kt_nw,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw,lat/long
48783,48783,AL012015,ANA,2015-05-10 10:00:00,L,TS,33.8,-78.8,40,1002.0,...,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"33.8,-78.8"
48797,48797,AL022015,BILL,2015-06-16 16:45:00,L,TS,28.2,-96.7,50,997.0,...,30.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,"28.2,-96.7"
49028,49028,AL112015,JOAQUIN,2015-10-01 12:00:00,L,HU,23.1,-73.7,115,942.0,...,110.0,60.0,60.0,40.0,50.0,35.0,30.0,30.0,30.0,"23.1,-73.7"
49033,49033,AL112015,JOAQUIN,2015-10-02 16:00:00,L,HU,23.6,-74.8,110,940.0,...,120.0,80.0,80.0,70.0,70.0,45.0,45.0,45.0,45.0,"23.6,-74.8"
49035,49035,AL112015,JOAQUIN,2015-10-02 21:00:00,L,HU,24.1,-74.5,110,942.0,...,120.0,80.0,80.0,70.0,70.0,50.0,45.0,45.0,45.0,"24.1,-74.5"


In [17]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [18]:
address_list = []
address = ""
counter = 0
for row in Subset_df['lat/long']:
    address = geolocator.reverse(row)
    address_list.append(address)
    
    #if location = geolocator.reverse(HUR_df['lat/long']).apply(geolocator.reverse,timeout=1000000)

In [19]:
Subset_df['address'] = address_list

<ipython-input-19-ca604f7b1d00>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Subset_df['address'] = address_list


In [20]:
Subset_df.head(2)

,index,id,name,date,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,...,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw,lat/long,address
4,4,AL011851,UNNAMED,1851-06-25 21:00:00,L,HU,28.2,-96.8,80,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"28.2,-96.8","(Second Chain of Islands, Calhoun County, Texa..."
48,48,AL041851,UNNAMED,1851-08-23 21:00:00,L,HU,30.1,-85.7,100,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"30.1,-85.7","(Upper Grand Lagoon, Bay County, Florida, 3241..."


In [21]:
Subset_df.to_csv('Subset_HURDAT_reverse_geo.csv')

In [22]:
print(geolocator.reverse(HUR_df['lat/long'][110]))

Bimini, The Bahamas


In [ ]:
HUR_data.head(2)